In [8]:
driver = webdriver.Chrome(executable_path="C:/Users/Owner/Downloads/chromedriver.exe", options=option)

In [9]:
driver.get('https://www.shakeshack.com/locations/')

In [5]:
shakeshack_state_element=driver.find_elements(By.XPATH,"//div[@class='row js-location-heading']")

In [6]:

len(shakeshack_state_element)

45

In [10]:
shakeshack_store_details_usa_dict={'store_name':[],'store_address':[],'store_direction_link':[]}

n=3
for i in shakeshack_state_element:
    #By default, Alabama is expanded when we land on locations page and hence we are ignoring n=1. there is a difference of 2 between consecutive n's and hence we set n=n+2.
    #The code below expands all states prior to extracting store details
    #Only 30 out of 45 locations are US states and hence setting the limit shown below
    if shakeshack_state_element.index(i) < 30:
        #print(n)
        state_element=driver.find_element(By.XPATH,'//*[@id="main"]/div/div/div[1]/div[{}]/div/div/div[2]/h3'.format(n))
        driver.execute_script("arguments[0].click();", state_element)
        state_stores=driver.find_elements(By.XPATH,'//div[@class="address"]')
        n=n+2
    else:
        break

        
#Extracting name of store
store_name_element=driver.find_elements(By.XPATH,'//h4')

for store_name in store_name_element:
    if store_name_element.index(store_name) <=190:
        shakeshack_store_details_usa_dict['store_name'].append(store_name.text)
    else:
        break
        

#Extracting address line items for store
store_address_element=driver.find_elements(By.XPATH,"//div[@class='address']")

for address in store_address_element:
    if store_address_element.index(address) <=190:
        shakeshack_store_details_usa_dict['store_address'].append(address.text)
        
    else:
        break

#Extracting underlying link from the directions link to parse as the data is cleaner than the one presented in address
store_directions_element= driver.find_elements(By.XPATH,"//a[@target='_blank' and @class='js-dir']")

for direction in store_directions_element:
    if  store_directions_element.index(direction)<=190:
        if direction.get_attribute('href')!='':
            shakeshack_store_details_usa_dict['store_direction_link'].append(direction.get_attribute('href'))
        else:
            shakeshack_store_details_usa_dict['store_direction_link'].append(direction.get_attribute('data-location'))
    else:
        break
    
        

driver.quit()              


In [490]:
#Creating a dataframe using the created dictionary
shakeshack_store_details_df=pd.DataFrame(shakeshack_store_details_usa_dict)

In [491]:
#Standardizing the links as some links started with https:// while others didn't
shakeshack_store_details_df['link_cleaned']=shakeshack_store_details_df['store_direction_link'].where(~shakeshack_store_details_df['store_direction_link'].str.contains('='), shakeshack_store_details_df['store_direction_link'].str.split('=').str[1])

In [492]:
#Creating Zipcode field
shakeshack_store_details_df['Zipcode']=shakeshack_store_details_df['link_cleaned'].str.split('+').str[-1]

In [493]:
#Cleaning Zipcode data
shakeshack_store_details_df['Zipcode']=shakeshack_store_details_df['Zipcode'].where(~shakeshack_store_details_df['Zipcode'].str.contains('%'), shakeshack_store_details_df['Zipcode'].str.split('%').str[0])

In [497]:
#Creating datframe with Store and Zipcode Infomtion which will be used in main script to aggregate stores info at a city level
shakeshack_store_zipcodes_df=shakeshack_store_details_df[['store_name','Zipcode']]

In [498]:
#Droopping zipcodes that are not 5 digits in length. Some links didn't have a zipcode and hence have to exlcude them from the results
shakeshack_store_zipcodes_df=shakeshack_store_zipcodes_df[shakeshack_store_details_df.Zipcode.str.len() ==5].reset_index(drop=True)

In [499]:
#vieweing final shakeshack storesdata frame
shakeshack_store_zipcodes_df

,store_name,Zipcode
0,"Birmingham, AL",35243
1,"Fashion Square, AZ",85251
2,"Kierland Commons, AZ",85254
3,"Phoenix Uptown Plaza, AZ",85012
4,"SanTan Village, Gilbert, AZ",89295
...,...,...
167,"Logan Circle, Washington, D.C.",20005
168,"Nationals Park, Washington, D.C.",20003
169,"Navy Yard, Washington, D.C.",20003
170,"The Wharf, Washington, D.C.",20024


In [ ]:
print("Shakeshack store data extraction complete! Created dataframe: shakeshack_store_zipcodes_df")